### retrieveTurnoverNumber

This script is designed to fetch turnover numbers and molecular weights for specified enzymes 
from the BRENDA database using their EC (Enzyme Commission) numbers. It utilizes the BRENDA 
SOAP web service to retrieve this biochemical data. The script requires the following:
- A client configured for accessing the BRENDA SOAP API.
- An input Excel file named "EC_Number_to_Reactions.xlsx" that contains the EC numbers.

#### Key Features:
- Fetch kcat (turnover number) and MW (molecular weight) for each enzyme listed in the Excel file.
- Outputs the results to a new sheet within the same Excel file, facilitating easy comparison and analysis.

#### Requirements:
- pandas library for DataFrame operations.
- zeep library for SOAP web service interaction.
- An account with BRENDA for necessary credentials.

#### Usage:
1. Ensure that the "EC_Number_to_Reactions.xlsx" file is present in the specified directory and contains a column 'EC Number'. (please check the code below this box)
2. Configure the client with appropriate credentials (parameters).
3. Run the script to append the fetched data into the Excel file under a new sheet named 'BRENDA Results+name_of_organism '.

Note: This script assumes that the user has already set up and authenticated a client for the BRENDA SOAP API.
"""

In [ ]:
import pandas as pd

# Read the excel file (model)
file_path = r".../Chinese Hamster Network Reconstruction.xlsx"
sheet_name = "Attributes"

df = pd.read_excel(file_path, sheet_name=sheet_name)

ec_to_reaction = {}

# List up the EC numbers and corresponding reactions in the model
for index, row in df.iterrows():
    ec_numbers = str(row['EC Number']).split(", ")
    for ec_number in ec_numbers:
        if ec_number not in ec_to_reaction:
            ec_to_reaction[ec_number] = [row['Reaction']]
        else:
            if row['Reaction'] not in ec_to_reaction[ec_number]:
                ec_to_reaction[ec_number].append(row['Reaction'])

# Convert into dataframe
ec_list = []
reactions_list = []
for ec, reactions in ec_to_reaction.items():
    ec_list.append(ec)
    reactions_list.append(", ".join(reactions))

result_df = pd.DataFrame({
    'EC Number': ec_list,
    'Reactions': reactions_list
})

# Save the list of EC numbers in model as new excel sheet
output_file_path = r"C:\Users\user\Documents\DC\Manual curation_iCHO\EC_Number_to_Reactions.xlsx"
result_df.to_excel(output_file_path, index=False)

print("Finish! output path is:", output_file_path)


In [1]:
#!/usr/bin/python
from zeep import Client
import hashlib
import pandas as pd
from openpyxl import load_workbook



###for parameters
email = 'BRENDA ID'
password = "your password"
organism_list = ['Homo sapiens', 'Cricetulus griseus', 'Mus musculus']
# (example) Choose human information for fetching the kcat and MW
organism = organism_list[0]
# Initialize the global dataframe 'results_df'
results_df = pd.DataFrame({"EC Number":[], f"kcat Values_{organism}":[], "MW Values":[]})
                                            
wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
password = hashlib.sha256(f"{password}".encode("utf-8")).hexdigest()
client = Client(wsdl)

In [2]:
def get_max_turnover(kcat_results):
    data_list = []
    for item in kcat_results:  
        data_dict = {
            'substrate': item.substrate,
            'organism': item.organism,
            'turnoverNumber': item.turnoverNumber,
            'ecNumber': item.ecNumber,
        }
        data_list.append(data_dict)
    
    df_tmp = pd.DataFrame(data_list)
    
    df_tmp['turnoverNumber'] = pd.to_numeric(df_tmp['turnoverNumber'], errors='coerce')
    
    max_value = df_tmp['turnoverNumber'].max()
    
    return max_value


In [3]:
def get_MW(mw_results):
    data_list_mw = []
    for item in mw_results:  
        data_dict_mw = {
            'molecularWeight': item.molecularWeight,
            'organism': item.organism,
            'ecNumber': item.ecNumber,
        }
        data_list_mw.append(data_dict_mw)
    
    df_tmp = pd.DataFrame(data_list_mw)
    

    df_tmp['molecularWeight'] = pd.to_numeric(df_tmp['molecularWeight'], errors='coerce')
    

    MW_value = df_tmp['molecularWeight'].min()
    
    return MW_value


In [9]:

def get_kcat_and_mw(client, ec_number, organism):
    global results_df  
    # Parameters for API calls and data processing
    parameters = (f"{email}",password,f"ecNumber*{ec_number}","turnoverNumber*", "turnoverNumberMaximum*", "substrate*", "commentary*",f"organism*{organism}", "ligandStructureId*", "literature*")
    parameters_mw = (f"{email}",password,f"ecNumber*{ec_number}", "molecularWeight*", "molecularWeightMaximum*", "commentary*", f"organism*{organism}", "literature*")
    
    try:
        kcat_results = client.service.getTurnoverNumber(*parameters)
        mw_results = client.service.getMolecularWeight(*parameters_mw)
        max_value = get_max_turnover(kcat_results)
        MW_value = get_MW(mw_results)
        
        print(f"EC Number: {ec_number}")
        print(f"kcat Values_max: {max_value}")
        print(f"MW Values: {MW_value}")
        print("------")
        
        # Add new row to the DataFrame
        new_row = pd.DataFrame({"EC Number": [ec_number], f"kcat Values_{organism}": [max_value], "MW Values": [MW_value]})
        results_df = pd.concat([results_df, new_row], ignore_index=True)

    except Exception as e:
        print(f"Error retrieving data for EC Number {ec_number}: {e}")

# Read Excel file and save results
input_file_path = r"C:\Users\user\Documents\DC\Manual curation_iCHO\EC_Number_to_Reactions.xlsx"
df = pd.read_excel(input_file_path)

# Run the function for each EC Number
for ec_number in df['EC Number']:
    get_kcat_and_mw(client, ec_number, organism)  

# Save the results to a new sheet in the Excel file
with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    results_df.to_excel(writer, sheet_name=f"BRENDA Results_{organism}", index=False)

print("The BRENDA search results have been saved to a new sheet in the Excel file.")

EC Number: 2.6.1.44
kcat Values_max: 45.0
MW Values: 42700
------
EC Number: 4.3.2.1
kcat Values_max: 34.5
MW Values: 49000
------
Error retrieving data for EC Number 6.3.4.5: 'turnoverNumber'
EC Number: 3.5.1.1
kcat Values_max: 3.19
MW Values: 12000
------
Error retrieving data for EC Number 3.5.1.38: 'turnoverNumber'
Error retrieving data for EC Number 3.5.5.4: 'turnoverNumber'
EC Number: 6.3.5.4
kcat Values_max: 1.8
MW Values: 64000
------
EC Number: 2.3.1.17
kcat Values_max: 0.035
MW Values: 33900
------
Error retrieving data for EC Number 1.4.1.21: 'turnoverNumber'
Error retrieving data for EC Number 1.4.3.16: 'turnoverNumber'
Error retrieving data for EC Number 1.4.3.2: 'turnoverNumber'
Error retrieving data for EC Number 2.6.1.1: 'turnoverNumber'
EC Number: 1.4.3.1
kcat Values_max: 81.3
MW Values: 37000
------
Error retrieving data for EC Number 1.4.3.15: 'turnoverNumber'
Error retrieving data for EC Number nan: 'turnoverNumber'
Error retrieving data for EC Number 3.5.1.14: 'mol

Error retrieving data for EC Number 2.3.1.1: 'turnoverNumber'
Error retrieving data for EC Number 1.8.4.8: 'turnoverNumber'
Error retrieving data for EC Number 1.1.1.310: 'turnoverNumber'
EC Number: 1.8.2.1
kcat Values_max: 50.0
MW Values: 52000
------
Error retrieving data for EC Number 1.13.11.53: 'turnoverNumber'
EC Number: 1.5.3.16 
kcat Values_max: 7.55
MW Values: 61900
------
EC Number: 3.2.1.52
kcat Values_max: 2.9333
MW Values: 26000
------
EC Number: 3.2.1.183
kcat Values_max: 11.8
MW Values: 80000
------
Error retrieving data for EC Number 5.1.3.14: 'turnoverNumber'
Error retrieving data for EC Number 5.1.3.7: 'turnoverNumber'
EC Number: 3.6.1.15
kcat Values_max: 0.0086
MW Values: 20500
------
Error retrieving data for EC Number 3.6.1.3: 'turnoverNumber'
EC Number: 3.6.1.5
kcat Values_max: 0.119
MW Values: 60500
------
Error retrieving data for EC Number 3.6.1.8: 'turnoverNumber'
Error retrieving data for EC Number 3.6.3.1: 'turnoverNumber'
Error retrieving data for EC Number

EC Number: 2.7.1.40
kcat Values_max: 1182.0
MW Values: -999
------
EC Number: 1.1.1.8
kcat Values_max: 240.0
MW Values: 35000
------
Error retrieving data for EC Number 2.7.1.2: 'turnoverNumber'
Error retrieving data for EC Number 5.3.1.1: 'turnoverNumber'
Error retrieving data for EC Number 4.1.2.19: 'turnoverNumber'
Error retrieving data for EC Number 1.2.1.21: 'turnoverNumber'
Error retrieving data for EC Number 1.2.3.5: 'turnoverNumber'
Error retrieving data for EC Number 2.7.1.165: 'turnoverNumber'
Error retrieving data for EC Number 2.7.1.31: 'turnoverNumber'
Error retrieving data for EC Number 1.1.1.79: 'molecularWeight'
Error retrieving data for EC Number 5.3.1.22: 'turnoverNumber'
Error retrieving data for EC Number 4.1.1.40: 'turnoverNumber'
Error retrieving data for EC Number 1.1.1.26: 'turnoverNumber'
Error retrieving data for EC Number 1.1.1.29: 'turnoverNumber'
Error retrieving data for EC Number 1.1.1.81: 'turnoverNumber'
EC Number: 3.1.3.18
kcat Values_max: 19
MW Values

Error retrieving data for EC Number 2.-.-.-: 'turnoverNumber'
EC Number: 2.4.2.26
kcat Values_max: 0.0363
MW Values: -999
------
Error retrieving data for EC Number 2.4.1.312: 'molecularWeight'
Error retrieving data for EC Number 2.7.1.183: 'turnoverNumber'
Error retrieving data for EC Number 2.4.2.-: 'turnoverNumber'
Error retrieving data for EC Number 2.4.1.-: 'turnoverNumber'
EC Number: 1.3.1.24
kcat Values_max: 94.8
MW Values: 21000
------
EC Number: 1.3.3.3
kcat Values_max: 16.0
MW Values: 37000
------
Error retrieving data for EC Number 4.99.1.1: 'turnoverNumber'
Error retrieving data for EC Number 4.4.1.17: 'turnoverNumber'
EC Number: 2.5.1.61
kcat Values_max: 0.25
MW Values: 25000
------
Error retrieving data for EC Number 4.3.1.8: 'turnoverNumber'
Error retrieving data for EC Number 1.14.99.3: 'turnoverNumber'
EC Number: 4.2.1.24
kcat Values_max: -999
MW Values: -999
------
EC Number: 1.3.3.4
kcat Values_max: 5.95
MW Values: 32000
------
EC Number: 4.2.1.75
kcat Values_max: 12

Error retrieving data for EC Number 1.5.1.37: 'turnoverNumber'
Error retrieving data for EC Number 1.5.1.45: 'turnoverNumber'
EC Number: 3.1.3.74
kcat Values_max: 1.52
MW Values: 25000
------
EC Number: 2.7.1.35
kcat Values_max: 6.08
MW Values: 35000
------
Error retrieving data for EC Number 1.2.3.8: 'turnoverNumber'
Error retrieving data for EC Number 1.4.3: 'turnoverNumber'
Error retrieving data for EC Number 1.1.3.12: 'turnoverNumber'
Error retrieving data for EC Number 1.11.1.11: 'turnoverNumber'
Error retrieving data for EC Number 3.1.1.17: 'turnoverNumber'
Error retrieving data for EC Number 4.1.1.-: 'turnoverNumber'
EC Number: 3.1.1.25
kcat Values_max: 632.0
MW Values: 17500
------
Error retrieving data for EC Number 1.1.1.20: 'turnoverNumber'
EC Number: 1.8.5.1
kcat Values_max: 5.27
MW Values: 32000
------
Error retrieving data for EC Number 3.1.1.19: 'turnoverNumber'
EC Number: 1.14.15.16
kcat Values_max: 0.007
MW Values: 55000
------
Error retrieving data for EC Number 1.14.

Error retrieving data for EC Number 2.8.2.8: 'turnoverNumber'
EC Number: 2.7.7.30
kcat Values_max: 3.6
MW Values: 66600
------
Error retrieving data for EC Number 3.1.3.46: 'turnoverNumber'
Error retrieving data for EC Number 2.7.1.52: 'turnoverNumber'
EC Number: 4.2.1.47
kcat Values_max: 0.8
MW Values: 38700
------
Error retrieving data for EC Number 2.7.7.70: 'turnoverNumber'
Error retrieving data for EC Number 2.7.1.7: 'turnoverNumber'
EC Number: 2.7.1.3
kcat Values_max: 11.1
MW Values: 32730
------
Error retrieving data for EC Number 2.7.7.22: 'turnoverNumber'
EC Number: 5.3.1.8
kcat Values_max: 3
MW Values: 46520
------
EC Number: 2.7.1.105
kcat Values_max: 0.696
MW Values: -999
------
EC Number: 5.4.2.8
kcat Values_max: 4.4
MW Values: 29000
------
Error retrieving data for EC Number 2.7.7.13: 'turnoverNumber'
Error retrieving data for EC Number 1.1.1.72: 'turnoverNumber'
Error retrieving data for EC Number 2.7.1.28: 'molecularWeight'
Error retrieving data for EC Number 3.2.1.50: 

Error retrieving data for EC Number 2.7.1.30: 'turnoverNumber'
Error retrieving data for EC Number 3.1.4.46: 'turnoverNumber'
Error retrieving data for EC Number 3.1.4.2: 'turnoverNumber'
Error retrieving data for EC Number 1.1.1.94: 'turnoverNumber'
Error retrieving data for EC Number  2.7.8.5 : 'turnoverNumber'
Error retrieving data for EC Number 2.7.8.11: 'turnoverNumber'
Error retrieving data for EC Number  1.14.99.62: 'turnoverNumber'
Error retrieving data for EC Number 2.3.1.67: 'turnoverNumber'
EC Number: 2.7.7.14
kcat Values_max: 0.26
MW Values: 50000
------
Error retrieving data for EC Number EC 3.1.3.27 : 'turnoverNumber'
Error retrieving data for EC Number 2.7.1.68: 'turnoverNumber'
Error retrieving data for EC Number 2.7.1.67: 'turnoverNumber'
Error retrieving data for EC Number EC:4.1.1.65: 'turnoverNumber'
Error retrieving data for EC Number 2.7.8.29: 'turnoverNumber'
Error retrieving data for EC Number 4.2.3.2: 'molecularWeight'
Error retrieving data for EC Number 2.1.1.

Error retrieving data for EC Number 2.5.1.39: 'turnoverNumber'
Error retrieving data for EC Number 4.2.1.-: 'turnoverNumber'
Error retrieving data for EC Number 2.1.1.163: 'turnoverNumber'
Error retrieving data for EC Number 2.1.1.201: 'turnoverNumber'
Error retrieving data for EC Number 1.3.99.6: 'turnoverNumber'
Error retrieving data for EC Number 1.14.99.38: 'turnoverNumber'
Error retrieving data for EC Number 1.1.1.181: 'turnoverNumber'
Error retrieving data for EC Number 1.14.13.15: 'turnoverNumber'
Error retrieving data for EC Number 1.14.13.99: 'turnoverNumber'
Error retrieving data for EC Number 1.14.13.98: 'turnoverNumber'
Error retrieving data for EC Number 1.14.13.17: 'turnoverNumber'
Error retrieving data for EC Number 1.14.13.100: 'turnoverNumber'
Error retrieving data for EC Number 1.14.13.95: 'turnoverNumber'
Error retrieving data for EC Number 3.1.2.27: 'turnoverNumber'
Error retrieving data for EC Number 2.3.1.15: 'turnoverNumber'
Error retrieving data for EC Number 3.

EC Number: 3.2.1.21
kcat Values_max: 17.6
MW Values: 46500
------
Error retrieving data for EC Number 3.1.3.60: 'turnoverNumber'
Error retrieving data for EC Number 3.1.2.3: 'turnoverNumber'
EC Number: 3.1.1.7
kcat Values_max: 6500.0
MW Values: -999
------
Error retrieving data for EC Number 1.14.99.19: 'turnoverNumber'
Error retrieving data for EC Number 2.7.8.1: 'turnoverNumber'
Error retrieving data for EC Number 1.1.1.101: 'turnoverNumber'
Error retrieving data for EC Number 2.3.1.51: 'turnoverNumber'
EC Number: 2.7.7.41
kcat Values_max: 8.0
MW Values: 53226
------
Error retrieving data for EC Number 2.3.1.6: 'turnoverNumber'
EC Number: 3.1.3.75
kcat Values_max: 2.27
MW Values: 29700
------
Error retrieving data for EC Number 2.7.7.15: 'turnoverNumber'
EC Number: 2.7.1.32
kcat Values_max: 91.1
MW Values: 45000
------
Error retrieving data for EC Number 2.7.1.29: 'turnoverNumber'
EC Number: 2.7.1.82
kcat Values_max: 0.00016
MW Values: 44000
------
EC Number: 3.5.1.23
kcat Values_max

In [ ]:
for ec_number in df['EC Number']:
    get_kcat_and_mw(client, ec_number, organism_list[1])  # 
    
# Save the results to a new sheet in the Excel file
with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    results_df.to_excel(writer, sheet_name="BRENDA Results_CHO, index=False)

In [ ]:
for ec_number in df['EC Number']:
    get_kcat_and_mw(client, ec_number, organism_list[2])  # 
    
# Save the results to a new sheet in the Excel file
with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    results_df.to_excel(writer, sheet_name="BRENDA Results_mouse", index=False)

Error retrieving data for EC Number 2.6.1.44: 'turnoverNumber'
Error retrieving data for EC Number 4.3.2.1: 'turnoverNumber'
Error retrieving data for EC Number 6.3.4.5: 'turnoverNumber'
Error retrieving data for EC Number 3.5.1.1: 'turnoverNumber'
Error retrieving data for EC Number 3.5.1.38: 'turnoverNumber'
Error retrieving data for EC Number 3.5.5.4: 'turnoverNumber'
Error retrieving data for EC Number 6.3.5.4: 'turnoverNumber'
Error retrieving data for EC Number 2.3.1.17: 'turnoverNumber'
Error retrieving data for EC Number 1.4.1.21: 'turnoverNumber'
Error retrieving data for EC Number 1.4.3.16: 'turnoverNumber'
Error retrieving data for EC Number 1.4.3.2: 'turnoverNumber'
Error retrieving data for EC Number 2.6.1.1: 'turnoverNumber'
EC Number: 1.4.3.1
kcat Values_max: 112.1
MW Values: 37620
------
Error retrieving data for EC Number 1.4.3.15: 'turnoverNumber'
Error retrieving data for EC Number nan: 'turnoverNumber'
EC Number: 3.5.1.14
kcat Values_max: 0.6
MW Values: 140000
-----

Error retrieving data for EC Number 1.8.4.8: 'turnoverNumber'
Error retrieving data for EC Number 1.1.1.310: 'turnoverNumber'
Error retrieving data for EC Number 1.8.2.1: 'turnoverNumber'
EC Number: 1.13.11.53
kcat Values_max: 17.1
MW Values: 21390
------
EC Number: 1.5.3.16 
kcat Values_max: 4.8
MW Values: 61852
------
Error retrieving data for EC Number 3.2.1.52: 'turnoverNumber'
Error retrieving data for EC Number 3.2.1.183: 'turnoverNumber'
Error retrieving data for EC Number 5.1.3.14: 'turnoverNumber'
Error retrieving data for EC Number 5.1.3.7: 'turnoverNumber'
Error retrieving data for EC Number 3.6.1.15: 'turnoverNumber'
Error retrieving data for EC Number 3.6.1.3: 'turnoverNumber'
Error retrieving data for EC Number 3.6.1.5: 'turnoverNumber'
Error retrieving data for EC Number 3.6.1.8: 'turnoverNumber'
Error retrieving data for EC Number 3.6.3.1: 'turnoverNumber'
Error retrieving data for EC Number 3.6.3.10: 'turnoverNumber'
Error retrieving data for EC Number 3.6.3.11: 'turno

Error retrieving data for EC Number 1.1.1.8: 'turnoverNumber'
Error retrieving data for EC Number 2.7.1.2: 'turnoverNumber'
Error retrieving data for EC Number 5.3.1.1: 'turnoverNumber'
Error retrieving data for EC Number 4.1.2.19: 'turnoverNumber'
Error retrieving data for EC Number 1.2.1.21: 'turnoverNumber'
Error retrieving data for EC Number 1.2.3.5: 'turnoverNumber'
Error retrieving data for EC Number 2.7.1.165: 'turnoverNumber'
Error retrieving data for EC Number 2.7.1.31: 'turnoverNumber'
Error retrieving data for EC Number 1.1.1.79: 'turnoverNumber'
Error retrieving data for EC Number 5.3.1.22: 'turnoverNumber'
Error retrieving data for EC Number 4.1.1.40: 'turnoverNumber'
Error retrieving data for EC Number 1.1.1.26: 'turnoverNumber'
Error retrieving data for EC Number 1.1.1.29: 'turnoverNumber'
Error retrieving data for EC Number 1.1.1.81: 'turnoverNumber'
Error retrieving data for EC Number 3.1.3.18: 'molecularWeight'
Error retrieving data for EC Number 1.14.18.2: 'turnoverN

Error retrieving data for EC Number 2.4.1.312: 'turnoverNumber'
Error retrieving data for EC Number 2.7.1.183: 'turnoverNumber'
Error retrieving data for EC Number 2.4.2.-: 'turnoverNumber'
Error retrieving data for EC Number 2.4.1.-: 'turnoverNumber'
Error retrieving data for EC Number 1.3.1.24: 'turnoverNumber'
Error retrieving data for EC Number 1.3.3.3: 'turnoverNumber'
Error retrieving data for EC Number 4.99.1.1: 'turnoverNumber'
Error retrieving data for EC Number 4.4.1.17: 'turnoverNumber'
Error retrieving data for EC Number 2.5.1.61: 'turnoverNumber'
Error retrieving data for EC Number 4.3.1.8: 'turnoverNumber'
Error retrieving data for EC Number 1.14.99.3: 'turnoverNumber'
Error retrieving data for EC Number 4.2.1.24: 'turnoverNumber'
EC Number: 1.3.3.4
kcat Values_max: 0.124
MW Values: 65000
------
Error retrieving data for EC Number 4.2.1.75: 'turnoverNumber'
Error retrieving data for EC Number 4.1.1.37: 'turnoverNumber'
Error retrieving data for EC Number 3.2.1.23: 'turnov

In [7]:
 organism_list[1]

'Cricetulus griseus'